In [1]:
import carla
import random
import cv2 #to work with images from cameras
import numpy as np

In [2]:
client=carla.Client('localhost',2000)

# load_world(self, map_name, reset_settings=True, map_layers=carla.MapLayer.All)

    Creates a new world with default settings using map_name map. All actors in the current world will be destroyed.

In [3]:
# It does not necessary to load again once it download

client.load_world('Town05')

In [4]:
world=client.get_world()
spawn_points=world.get_map().get_spawn_points()

In [5]:
vehicle_bp=world.get_blueprint_library().filter('*mini*')

In [5]:
vehicle_bp[1]

In [6]:
start_point=random.choice(spawn_points)
vehicle=world.try_spawn_actor(vehicle_bp[0], start_point)

In [39]:
spectator=world.get_spectator()
start_point.location.z=start_point.location.z+1
spectator.set_transform(start_point)

vehicle.set_autopilot(True)

# set_attribute(self, id, value) :
    If the id attribute is modifiable, changes its value to value.
    id (str) - The identifier for the attribute that is intended to be changed.
    value (str) - The new value for said attribute.

In [51]:
CAMERA_POS_Z=1.6 #1.6m up from the ground
CAMERA_POS_X=0.9 #0.9m forward

camera_bp=world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') #640 pixels tall
camera_bp.set_attribute('image_size_y', '360') #360 pixels wide

camera_init_trans=carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
# camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
camera = world.spawn_actor(camera_bp,camera_init_trans)

# carla.Image
    Class that defines an image of 32-bit BGRA colors that will be used as initial data retrieved by camera sensors. There are different camera sensors (currently three, RGB, depth and semantic segmentation) and each of these makes different use for the images.
    
    -fov (float - degrees):Horizontal field of view of the image.
    -height (int):Image height in pixels.
    -width (int):Image width in pixels.
    -raw_data (bytes):Flattened array of pixel data, use reshape to create an image array.

# image returns dict type???

# get_attribute(self, id) :
    Returns the actor's attribute with id as identifier if existing.

In [52]:
def camera_callback(image, data_dict):
    data_dict['image']=np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
image_w=camera_bp.get_attribute('image_size_x').as_int()
image_h=camera_bp.get_attribute('image_size_y').as_int()

camera_data={'image':np.zeros((image_h, image_w, 4))}

In [57]:
camera.listen(lambda image:image.save_to_disk('out/%06d.png'% image.frame))

# computer vision
    the way of teaching intelligence to machine and making them see things like humans. What allows computers to see and process visual data like humans, analyzing images to produce useful information

# OpenCV
    vision library of programming functions mainly aimed at real time computer vision. it can be showed a digital image seen by a computer in matrix
    
# namedWindow() :
    is used to create a window with a suitable name and size to display images and videos on the screen.
    
# WINDOW_NORMAL : 
    it enables you to resize the window
   
# WINDOW_AUTOSIZE: 
    it adjusts automatically the window size to fit the displayed image (see imshow ), and you cannot change the window size manually

In [54]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

while(True):
    if cv2.waitKey(1) & 0xFF==ord('x'):
        cv2.destroyAllWindows()
        break

In [44]:
# cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
# cv2.imshow('RGB Camera', camera_data['image'])
# cv2.waitKey(1)

# while cv2.waitKey(0)==-1:
#     cv2.imshow('RGB Camera', camera_data['image'])
#     if cv2.waitKey(1)==ord('q'):
#         break

In [61]:
camera.stop() #this is the opposite of camera.listen

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
    
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [62]:
cv2.destroyAllWindows()